## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-15-50-33 +0000,bbc,"'Long and painful nightmare finally over,' Tru...",https://www.bbc.com/news/articles/c709jxxrrvlo...
1,2025-10-13-15-49-15 +0000,nyt,"Small Plane Crashes on Massachusetts Highway, ...",https://www.nytimes.com/2025/10/13/us/highway-...
2,2025-10-13-15-45-00 +0000,wsj,LendingTree CEO Dies in Accident,https://www.wsj.com/business/lendingtree-ceo-d...
3,2025-10-13-15-44-26 +0000,nyt,Live Updates: Hostages and Palestinian Prisone...,https://www.nytimes.com/live/2025/10/12/world/...
4,2025-10-13-15-44-25 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/12/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,47
203,china,22
6,israel,21
20,hostages,16
219,hamas,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-10-13-15-32-35 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...,128
215,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,126
18,2025-10-13-15-25-03 +0000,startribune,Trump arrives in Egypt for Gaza summit after u...,https://www.startribune.com/trump-arriving-in-...,114
168,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...,108
59,2025-10-13-13-11-27 +0000,nypost,"Dow futures soar 400 points after Trump, Vance...",https://nypost.com/2025/10/13/business/dow-fut...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,128,2025-10-13-15-32-35 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...
164,59,2025-10-13-06-16-39 +0000,nypost,Last living hostages all released after 738 da...,https://nypost.com/2025/10/13/world-news/last-...
85,52,2025-10-13-11-28-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
134,51,2025-10-13-09-06-41 +0000,nypost,Heartwarming moment shows Israeli hostage spea...,https://nypost.com/2025/10/13/world-news/mothe...
30,49,2025-10-13-14-55-52 +0000,nypost,JPMorgan to invest $10B in areas ‘critical’ to...,https://nypost.com/2025/10/13/business/jpmorga...
163,30,2025-10-13-06-21-00 +0000,bbc,"Boy, 11, shoots farm worker dead in tragic Sou...",https://www.bbc.com/news/articles/ce86dvr03zqo...
61,29,2025-10-13-13-06-20 +0000,nypost,Trump urges Israeli president to pardon Benjam...,https://nypost.com/2025/10/13/us-news/trump-ur...
29,26,2025-10-13-14-58-51 +0000,nyt,Nor’easter Brings Heavy Rain and Flooding as I...,https://www.nytimes.com/2025/10/12/us/noreaste...
250,26,2025-10-12-17-31-50 +0000,nypost,Father of Israeli-American hostage assumed mur...,https://nypost.com/2025/10/12/world-news/hosta...
223,25,2025-10-12-21-42-29 +0000,nypost,Zelensky suggests Tomahawk missiles could make...,https://nypost.com/2025/10/12/us-news/zelensky...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
